In [1]:
import urllib.request
from bs4 import BeautifulSoup

In [13]:
companies = ['Facebook','Alibaba']

In [20]:
def retrieve_links(companies):
    for company in companies:
        print(company)
        print('\n')
        
        quote_page = "https://news.google.com/news/search/section/q/$REPLACE/$REPLACE?hl=en&gl=US&ned=us"
        quote_page = quote_page.replace("$REPLACE", company)
        
        page = urllib.request.urlopen(quote_page)
        soup = BeautifulSoup(page, "html.parser")

        
        arias = soup.find_all('a')
        links = []
        titles = []
        
        #look for aria-level="2"
        for aria in arias:
            if aria.get('aria-level')=="2":
                if aria.get('href')[0:1]=="h": #if it is a https link
                    links.append(aria.get('href'))
                    titles.append(aria.string)

        links = list(dict.fromkeys(links))
        for i in range(0, 3):
            print(titles[i])
            print(links[i])
            print('\n')
        print('\n')
retrieve_links(companies)

Facebook


Facebook Pulls VR Shooter Demo From CPAC After Backlash
https://kotaku.com/facebook-pulls-vr-shooter-demo-from-cpac-after-backlash-1823298308


This Facebook recruiting initiative turned a psychology major into a sought-after product manager
http://www.businessinsider.com/what-is-the-facebook-rotational-product-management-program-2018-2


Facebook and Google Struggle to Squelch 'Crisis Actor' Posts
https://www.nytimes.com/2018/02/23/technology/trolls-step-ahead-facebook-youtube-florida-shooting.html




Alibaba


China's new e-commerce players take on Alibaba and JD.com
https://asia.nikkei.com/Business/Companies/China-s-new-e-commerce-players-take-on-Alibaba-and-JD.com


Ant Financial: Another Reason To Be Bullish On Alibaba
https://seekingalpha.com/article/4150017-ant-financial-another-reason-bullish-alibaba


Altaba Investors Hope to Cash In on Alibaba Stake
https://www.barrons.com/articles/altaba-investors-hope-to-cash-in-on-alibaba-stake-1519392524






In [15]:
#just viewing the page's elements
alibaba_page = "https://news.google.com/news/search/section/q/alibaba/alibaba?hl=en&gl=US&ned=us"
page = urllib.request.urlopen(alibaba_page)
soup = BeautifulSoup(page, "html.parser")
print(soup.prettify())

<!DOCTYPE doctype html>
<html dir="ltr" lang="en">
 <head>
  <base href="https://news.google.com/news/"/>
  <meta content="origin" name="referrer"/>
  <link href="https://news.google.com/search/section/q/alibaba/alibaba" rel="canonical"/>
  <meta content="initial-scale=1,minimum-scale=1,maximum-scale=1,user-scalable=no" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <meta content="Google News" name="application-name"/>
  <meta content="Google News" name="apple-mobile-web-app-title"/>
  <meta content="black" name="apple-mobile-web-app-status-bar-style"/>
  <meta content="no" name="msapplication-tap-highlight"/>
  <link href="//encrypted-tbn0.gstatic.com" rel="preconnect"/>
  <link href="//encrypted-tbn1.gstatic.com" rel="preconnect"/>
  <link href="//encrypted-tbn2.gstatic.com" rel="preconnect"/>
  <link href="//encrypted-tbn3.gstatic.com" rel="preconnect"/>
  <link href="//fonts.googleapis.com" rel="